a. Extraer la información del archivo.
  
  Para extraer la información del archivo, ocupamos pandas con rad_csv al archivo colocado en la raíz

  Imprimimos el DF para saber que está de forma correcta con el head y types.

In [2]:
import pandas as pd
import os

# Leer el archivo CSV.
df = pd.read_csv('country_vaccinations.csv')

# Mostrar los datos del DataFrame solo para asegurarnos que se cargó de forma
# correcta.
print("\n Contenido del dataframe: \n")
print(df.head)
print("\n Tipos de datos para el dataframe: \n")
print(df.dtypes)




 Contenido del dataframe: 

<bound method NDFrame.head of            country iso_code        date  total_vaccinations  \
0      Afghanistan      AFG  2021-02-22                 0.0   
1      Afghanistan      AFG  2021-02-23                 NaN   
2      Afghanistan      AFG  2021-02-24                 NaN   
3      Afghanistan      AFG  2021-02-25                 NaN   
4      Afghanistan      AFG  2021-02-26                 NaN   
...            ...      ...         ...                 ...   
86507     Zimbabwe      ZWE  2022-03-25           8691642.0   
86508     Zimbabwe      ZWE  2022-03-26           8791728.0   
86509     Zimbabwe      ZWE  2022-03-27           8845039.0   
86510     Zimbabwe      ZWE  2022-03-28           8934360.0   
86511     Zimbabwe      ZWE  2022-03-29           9039729.0   

       people_vaccinated  people_fully_vaccinated  daily_vaccinations_raw  \
0                    0.0                      NaN                     NaN   
1                    NaN      

b. Mostrar la estructura y tipos de datos de cada columna para identificar qué operaciones puedes realizar con cada una de ellas, asegurándote que las columnas con fechas sean del tipo datetime64.

Como ya hemos mostrado los tipos de datos contenidos en el DF, ahora podemos identificar la columna y cambiar el tipo de dato a datetime64 mediante to_date.

Imprimimos de nuevo los tipos de dato para corroborar que se haya realizado el cambio.


In [ ]:
# Se identifica la columna date y se convierte de object a DateTime usando
# to_datetime().
df['date'] = pd.to_datetime(df['date'])
print("\n Tipos de datos para el dataframe corregido: \n")
print(df.dtypes)




c. Determinar la cantidad de vacunas aplicadas de cada compañía (con base en cómo lo reporta cada país en la columna vaccines, en otras palabras, agrupe por vaccines y realice la sumatoria).


Para esto, creamos un df llamado df_separado para separar las marcas de las vacunas que están en una sola columna divididos por " , "

Posteriormente, quitaremos los espacios para evitarnos cualquier inconveniente al comparar los nombres.

Después calcularemos cuantas marcas hay por cada día con len() y lo guardamos en una nueva columna colocando un número entero.
Ahora, para calcular  la cantidad de vacunas por marca, sugerimos utilizar de forma equitativa la cantidad de vacunas por día, así que dividimos las vacunas diarias, entre el numero de marcas aplicadas ese día. Guardamos el resultado en una nueva columna llamada vacunas_por_marca_diario en el df_separado.

Finalmente para resolver este punto, agrupamos por el contenido de vaccine.
Para imprimir el resultado colocamos un for para que todas las marcas las vaya recorriendo y colocandolas en pantalla.

In [ ]:


# Crear DataFrame separado por marca de vacuna
df_separado = (
    # Dividir las marcas
    df.assign(vaccine=df['vaccines'].str.split(', '))
    # Crear una fila por cada marca
    .explode('vaccine')
    # Limpiar espacios
    .assign(vaccine=lambda x: x['vaccine'].str.strip())
)

# Calcular cuántas marcas hay por fila
df_separado['num_vacunas_por_dia'] = df_separado['vaccines'].str.split(', ').str.len()

# Distribuir equitativamente las vacunas diarias entre las marcas que aparecen
# ese día.
df_separado['vacunas_por_marca_diario'] = (
    df_separado['daily_vaccinations'] / df_separado['num_vacunas_por_dia']
).fillna(0)

# Agrupar por marca y sumar las vacunas distribuidas
vacunas_por_marca = df_separado.groupby('vaccine')['vacunas_por_marca_diario'].sum().sort_values(ascending=False)

print("\n Vacunas aplicadas por marca (distribución diaria equitativa) \n")

for marca, total in vacunas_por_marca.items():
    print(f"{marca}: {total:,.0f} vacunas \n")



d. Obtener la cantidad de vacunas aplicadas en todo el mundo.

Para este punto, ocupamos la función de sum() para obtener una variable llamada total_vacunas_diarias que tiene la sumatoria de daily vaccinations del df original.

Imprimimos el total.

In [ ]:
# Arriba calculamos la cantidad total de vacunas aplicadas.
total_vacunas_diarias = df['daily_vaccinations'].sum()
print(f"\nTotal diario mundial verificado: {total_vacunas_diarias:,.0f} vacunas")




e. Calcular el promedio de vacunas aplicadas por país

Para esto, ya tenemos el total de vacunas y solo nos falta obtener el total de paises, asi que agrupamos vacunas por país sumando la columna daily_vaccionations. Entonces podemos calcular el promedio, de la serie que acabamos de crear, sacamos la cantidad de paises con len() y podemos hacer la división correspondiente.
Colocamos en una impresión el total de vacunas aplicadas a nivel mundial y la cantidad de paises; finalmente colocamos el promedio por país


In [ ]:
# Ahora para sacar las vacunas promedio por país, ya que obtenermos el total de
# vacunas, obtenermos la cantidad de paises.
# Para calcular el total de vacunas por país, sumamos daily_vaccinations
total_vacunas_por_pais = df.groupby('country')['daily_vaccinations'].sum().sort_values(ascending=False)

# Calcular promedios
total_vacunas_mundial = total_vacunas_por_pais.sum()
cantidad_paises = len(total_vacunas_por_pais)
promedio_por_pais = total_vacunas_mundial / cantidad_paises

print(f"\n Total mundial de vacunas aplicadas: {total_vacunas_mundial:,.0f} Cantidad de países: {cantidad_paises}")
print(f"\n Promedio por país: {promedio_por_pais:,.0f} vacunas")



f. Determinar la cantidad de vacunas aplicadas el día 29/01/21 en todo el mundo.

Para resolver este inciso, debemos hacer la sumatoria del filtro donde el valor de fecha sea igual a la fecha indicada del 2021-01-29, realizamos una suma de los valores filtrados y esto nos arrojará un número entero que podemos imprimir.

In [ ]:

# Para saber cuantas vacunas a nivel mundial se aplicaron, debemos hacer una
# suma del resultado de un filtro de fecha específica al 29/01/21
vacunas_29_ene_2021 = df[df['date'] == '2021-01-29']['daily_vaccinations'].sum()
print(f"\n Vacunas aplicadas el 29/01/2021: {vacunas_29_ene_2021:,.0f} \n")



g. Crear un dataframe nuevo denominado conDiferencias que contenga los datos originales y una columna derivada (diferencias) con las diferencias de aplicación entre las columnas daily_vaccionations y daily_vaccionations_raw.  

Ahora crearemos el nuevo df indicado con la columna derivada, sin embargo nos percatamos que existen valores nulos y nos marcaría un error. Para evitar esta situación, colocaremos un 0 en caso de que no haya valor, es decir sea nulo.

Finalmente realizamos la operación de la resta y la imprimimos.

In [ ]:

# Crearemos un df llamado conDiferencias con una columna extra que se llave
# diferencias y sea la resta entre daily_vaccionations y daily_vaccionations_raw
# Como no todas las filas tienen un valor en la parte de daily_raw habrá un
# error al hacer la operación, por ende si hay valores nulos, considero
# conveniente manejarlos antes de calcular diferencias. Asumiremos que si está
# vacía la celda el valor el 0 porque no hubo vacunaciones de ese tipo ese día.
conDiferencias = df
conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'].fillna(0) - conDiferencias['daily_vaccinations_raw'].fillna(0)
print(f"\n El contenido de conDiferencias es: \n {conDiferencias['diferencias']}")



h. Obtener el periodo de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.

Para obtener estos datos, utilizaremos las funciones min() y max() guardadas en variables independientes y posteriormente haremos una resta. después le indicaremos que nos arroje el resultado en días ya que es una diferencia de tiempo.

Posteriormente y para poder imprimir de forma ordenada y más legible, haremos las divisiones pertinentes para saber en años meses y días.

Imprimimos el periodo total y su equivalencia en dias, meses y años.



In [ ]:
# Ahora buscamos obtener el periodo de tiempo entre el registro con fecha más
# reciente y el registro con fecha más antiguo, para lo cual usaremos min y
# max, y posteriormente los restaremos para saber la diferencia en días.
fecha_min = df['date'].min()
fecha_max = df['date'].max()
dias_totales = (fecha_max - fecha_min).days

# Para poder mostrar la información más digerible, necesitamos convertir días
# a años, meses y el remanente en días
años = dias_totales // 365
meses = (dias_totales % 365) // 30
dias = (dias_totales % 365) % 30

# Imprimiremos el periodo con un formato de días y en una segunda impresión
# colocaremos un formato de años, meses y su residuo en días.
print(f"\n Período total entre las fechas más distantes: {dias_totales} días")
print(f"\n Equivale a {años} años, {meses} meses y {dias} días")


i. Crear un dataframe nuevo denominado conCantidad que contenga los datos originales y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter , ).


Creamos el df, separamos por " , " y utilizamos un len para poder saber la cantidad de vacunas.
Mostramos las primeras filas para saber que se agregó de forma correcta.


In [ ]:
# Crear el nuevo DataFrame conCantidad con la columna canVac
conCantidad = df.assign(
    canVac=df['vaccines'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) and x != '' else 0)
)

# Mostrar las primeras filas para verificar que se agregó correctamente
print("\n Contenido de conCantidad: \n")
print(conCantidad.head())



j. Generar un dataframe denominado antes20 con todos los registros que se hayan realizado antes del 20 de diciembre de 2020.

Obtenemos del df original, la columna date y colocamos una condición de solo valores antes del 20 de dic de 2020, esto comparando la columna date. Al final agregamos un len() para saber cuántos registros hay y poder validar manualmente.


In [ ]:
# Para crear DataFrame con registros antes del 20-dic-2020 solo debemos
# indicarle la fecha para que sea menor, ya que hemos convertido previamente el
# valor para poder compararlo. Imprimimos la cantidad de registros.
antes20 = df[df['date'] < '2020-12-20']
print(f"\n Cantidad de registros antes del 20-dic-2020: {len(antes20):,}")


k. Obtener un dataframe denominado pfizer con todos los registros donde se haya utilizado la vacuna Pfizer.

Para este punto, el dataframe lo firltraremos utilizando la función contains(), pero es importante convertir a string el contenido de la columna "vacciones" para poder compararlo adecuadamente.

Una vez que tengamos el resultado, solo colocamos la cantidad de registros del df pfizer.


In [ ]:

# El requeriminto es crear un DataFrame con registros que contengan
# Pfizer/BioNTech, para lo cual haremos un str.contains para saber si está
# dentro de la columna vaccines
pfizer = df[df['vaccines'].str.contains('Pfizer/BioNTech', na=False)]
print("\n Contenido del dataframe: \n")
print(pfizer.head)
print(f"\n Cantidad de registros con Pfizer/BioNTech: {len(pfizer):,}")


l. Almacenar los dataframes generados (conDiferencias, conCantidad, antes20 y pfizer) en un archivo de Excel denominado resultadosReto.xlsx, donde cada dataframe ocupe una hoja diferente. Se recomienda ver la documentación de pd.ExcelWriter.

Exportaremos entonces los libros de excel separado una hoja para cada df. Lo haremos a través de to_excel.

Después de revisar los resultados, noté que esta función agrega una columna con un identificador, así que lo quité colocando index en false.

Al final, imprimí el tamaño del archivo para validar que no estuviera vacío y con eso verificar que la escritura ser realizó.

In [ ]:
# Por ultimo se requiere exportar a un libro de excel con una hoja por
# dataframe, para ello vamos a necesitar cargar con un to_excel. Me percaté que
# al principio pandas coloca una columna numerica adicional, así que para
# llevarnos el df fiel, colocaremos la etiqueta index en false.
with pd.ExcelWriter('resultadosReto.xlsx', engine='openpyxl') as writer:
    conDiferencias.to_excel(writer, sheet_name='conDiferencias', index=False)
    conCantidad.to_excel(writer, sheet_name='conCantidad', index=False)
    antes20.to_excel(writer, sheet_name='antes20', index=False)
    pfizer.to_excel(writer, sheet_name='pfizer', index=False)

# Verificación del archivo generado por medio del peso del archivo (que
# debería ser distinto de 0) y convertimos a MB para que sea más facil de
# entender.

tamaño_mb = os.path.getsize('resultadosReto.xlsx') / (1024 * 1024)
print(f"\n Archivo creado: resultadosReto.xlsx ({tamaño_mb:.2f} MB)")